<a href="https://colab.research.google.com/github/LIKHITA12/SENTIMENT-ANALYSIS-WEB-APP/blob/main/Sentiment_Analysis_Web_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Complete End-To-End ML Prototype: **SENTIMENT ANALYSIS WEB APP**

### Setup and Installations

In [ ]:
print("STEP 1: Installing libraries...")
!pip install -q flask pyngrok tensorflow numpy

STEP 1: Installing libraries...


### Train a TensorFlow/Keras model for sentiment analysis.

In [ ]:
print("\nSTEP 2: Training the Sentiment Analysis Model...")
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

# --- Model Parameters ---
vocab_size = 10000
max_length = 256
embedding_dim = 16

# --- Load and Preprocess Data ---
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

# --- Build and Train the Neural Network ---
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# We set verbose=0 to make the training log cleaner for this final version
model.fit(X_train, y_train, epochs=20, batch_size=512, validation_split=0.2, verbose=0)
print("✅ Model training complete.")

# --- Create helper functions for prediction ---
word_index = imdb.get_word_index()
word_index = {k:(v+3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

def preprocess_text(text):
    words = text.lower().split()
    tokens = [word_index.get(word, 2) for word in words]
    return pad_sequences([tokens], maxlen=max_length, padding='post')



STEP 2: Training the Sentiment Analysis Model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


✅ Model training complete.


### Define a complete Flask web application.

In [ ]:
print("\nSTEP 3: Defining the Flask Web Application...")
from flask import Flask, request, jsonify, render_template_string

app = Flask(__name__)

# --- HTML Template for the Web Page ---
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sentiment Analysis Demo</title>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');
        body { font-family: 'Inter', sans-serif; background-color: #f8f9fa; color: #212529; display: flex; justify-content: center; align-items: center; min-height: 100vh; margin: 0; }
        .container { max-width: 600px; width: 100%; padding: 30px; background-color: #ffffff; border-radius: 12px; box-shadow: 0 4px 20px rgba(0,0,0,0.05); text-align: center; }
        h1 { font-size: 28px; font-weight: 700; color: #343a40; margin-bottom: 10px; }
        p { color: #6c757d; margin-bottom: 25px; }
        textarea { width: calc(100% - 24px); padding: 12px; border: 1px solid #ced4da; border-radius: 8px; font-size: 16px; margin-top: 10px; resize: vertical; min-height: 100px; }
        textarea:focus { border-color: #80bdff; outline: 0; box-shadow: 0 0 0 .2rem rgba(0,123,255,.25); }
        button { display: block; width: 100%; padding: 12px; background-color: #007bff; color: white; border: none; border-radius: 8px; font-size: 16px; font-weight: 600; cursor: pointer; margin-top: 20px; transition: background-color 0.2s; }
        button:hover { background-color: #0056b3; }
        #result { margin-top: 25px; padding: 15px; border-radius: 8px; font-size: 20px; font-weight: 600; display: none; }
        #result.positive { background-color: #d4edda; color: #155724; border: 1px solid #c3e6cb;}
        #result.negative { background-color: #f8d7da; color: #721c24; border: 1px solid #f5c6cb;}
    </style>
</head>
<body>
    <div class="container">
        <h1>Live Movie Review Sentiment Analysis</h1>
        <p>Enter a movie review below to see a real-time prediction from a neural network.</p>
        <form id="sentiment-form">
            <textarea id="review_text" rows="5" placeholder="e.g., 'The movie was absolutely fantastic! The acting was brilliant.'"></textarea>
            <button type="submit">Analyze Sentiment</button>
        </form>
        <div id="result"></div>
    </div>
    <script>
        document.getElementById('sentiment-form').addEventListener('submit', async function(event) {
            event.preventDefault();
            const resultDiv = document.getElementById('result');
            const reviewText = document.getElementById('review_text').value;
            resultDiv.style.display = 'block';
            resultDiv.innerText = 'Analyzing...';
            resultDiv.className = '';
            const response = await fetch('/predict', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ review_text: reviewText })
            });
            const data = await response.json();
            resultDiv.innerText = `Prediction: ${data.prediction} (${data.confidence}%)`;
            resultDiv.className = data.prediction === 'Positive' ? 'positive' : 'negative';
        });
    </script>
</body>
</html>
"""

@app.route("/")
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    review_text = data.get("review_text")
    if not review_text:
        return jsonify({"error": "No text provided"}), 400
    # Add verbose=0 to prevent prediction logs from cluttering the output
    processed_text = preprocess_text(review_text)
    raw_prediction = model.predict(processed_text, verbose=0)[0][0]
    if raw_prediction > 0.5:
        sentiment = "Positive"
        confidence = f"{raw_prediction * 100:.1f}"
    else:
        sentiment = "Negative"
        confidence = f"{(1 - raw_prediction) * 100:.1f}"
    return jsonify({"prediction": sentiment, "confidence": confidence})
print("✅ Flask application defined.")



STEP 3: Defining the Flask Web Application...
✅ Flask application defined.


### Launch the Web Server with ngrok


In [ ]:
print("\nSTEP 4: Launching the application...")
from pyngrok import ngrok

# --- ngrok Configuration ---
# This is the authtoken you provided.
authtoken = "2yH2jekFMKmgnfzke5aUnzuuYsn_V2Gy4FSV2p4EL1A1PCtQ"
ngrok.set_auth_token(authtoken)

# --- Launch the server ---
# This opens a public tunnel to your Flask app
public_url = ngrok.connect(5000, proto="http")
print("\n" + "="*80)
print(f"✅ PROTOTYPE IS LIVE! Click this temporary URL to access it -> {public_url}")
print("="*80)

# This will run the Flask app and keep the cell running until you manually stop it.
app.run(port=5000)


STEP 4: Launching the application...

✅ PROTOTYPE IS LIVE! Click this temporary URL to access it -> NgrokTunnel: "https://c424-34-145-168-171.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 17:02:14] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 17:02:15] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 17:02:25] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 17:02:37] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 17:03:20] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 17:03:50] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 17:04:11] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 17:04:21] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 17:05:05] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 17:05:21] "GET / HTTP/1.1" 200